# Uniswap Deposits like an Baller

In [ ]:
import os
import math
import numpy as np
from termcolor import colored

In [ ]:
from uniswappy import *

In [ ]:
user_nm = 'user0'
eth_amount = 1000
usdc_amount = 1000000

#### SwapDeposit (incl. fee):

* A ```SwapDeposit``` is where a certain amount of a specific token is deposited into the LP under one operation
* Includes two steps:
    * (step 1) perform approx. 50% swap for opposing token
    * (step 2) using amt from step 1, perform 1:1 deposit
* A portion of the incoming funds are swapped to achieve equal portions of both assets
* These portions are then deposited into the LP
* To ensure all the funds are deposited, we must determine the portion ($\alpha$) of $s_{in}$ that must first get swapped

> $ \Delta x =\Delta x_{swap}  + \Delta x_{deposit}$

> $ \Delta x =\alpha \Delta x   + \Delta x_{deposit}$


#### Follows this system of equations:

> (Eq. 1) $\Delta y = \frac{997 y \alpha \Delta x }{1000x + 997\alpha\Delta x }$

> (Eq. 2) $\Delta x = \alpha\Delta x   + \frac{\Delta y(x + \alpha\Delta x )}{y - \Delta y}$

where 
* $\Delta x$ -> amt token in 
* $\Delta y$ -> amt opposing token out after swap
* $\alpha$ -> portion of $\Delta x$ swapped in
* x -> reserve0
* y -> reserve1

### Let's highlight why the above considerations are important ...

In [ ]:
user_nm = 'user_intro'
fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)
init_price = UniV3Utils.encodePriceSqrt(1, 10)

tkn = ERC20("TKN", "0x111")
dai = ERC20("DAI", "0x09")

exchg_data = UniswapExchangeData(tkn0 = tkn, tkn1 = dai, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("TKN pool factory", "0x2")
lp = factory.deploy(exchg_data)
lp.initialize(init_price)
out_v3 = lp.mint(user_nm, lwr_tick, upr_tick, 3162.2776601683795)

dai_amount = lp.get_reserve(dai)

print('***\nInitial LP\n***')
lp.summary()

s_in = 100
alpha = 0.5
s_out = Swap().apply(lp, tkn, user_nm, alpha*s_in)

print('***\nLP post step 1\n***')
lp.summary()

sqrt_P = lp.slot0.sqrtPriceX96/2**96
balance1 = s_in - alpha*s_in
liq = balance1*sqrt_P

out = lp.mint(user_nm, lwr_tick, upr_tick, liq)

print('***\nLP post step 2\n***')
lp.summary() 


print('Given {} initial TKN:'.format(s_in))
print('  (step 1) {} TKN must first get swapped for {} '.format(alpha*s_in, s_out[1]))
print('  (step 2) The received TKN gets deposited along with the remaining {} TKN'.format(balance1))

print('\nTotal deposited is {:.6f} + {:.6f} = {:.6f} TKN:'.format(alpha*s_in,  balance1, alpha*s_in + balance1))
print('However, we have {} unaccounted DAI which need to be considered when using a 50/50 split'.format(colored(str(dai_amount-lp.get_reserve(dai)), 'red', attrs=['bold'])))

### Let's now address this problem ...

In [ ]:
tkn = ERC20("TKN", "0x111")
dai = ERC20("DAI", "0x09")

exchg_data = UniswapExchangeData(tkn0 = tkn, tkn1 = dai, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("TKN pool factory", "0x2")
lp = factory.deploy(exchg_data)
lp.initialize(init_price)
out_v3 = lp.mint(user_nm, lwr_tick, upr_tick, 3162.2776601683795)

s_in = 100
alpha = 0.5
y = lp.get_reserve(dai)
x = lp.get_reserve(tkn)

#### Plug above into equation (1), and see how many TKN we get when 50% of ETH is swapped in for step (1)

In [ ]:
s_out = (997*alpha*s_in*y)/(1000*x + 997*alpha*s_in)
print('For {} TKN, we get {:.2f} DAI with a 50% portion'.format(s_in, s_out))

#### Now, lets check how many ETH gets SwapDeposited in when 50% of ETH is swapped in for step (1)

In [ ]:
a1_out = alpha*s_in + s_out*(x + alpha*s_in)/(y - s_out)
print('Instead of {} TKN, we get {:.2f} TKN under a 50% portion'.format(s_in, a1_out))

We can see that there is an imbalance in the system under a 50% distribution for step (1); 
* we need to solve the system above for $\alpha$ to get the proper distribution
* plug (1) into (2) and we get:

$\Delta x = \alpha\Delta x + \left(\frac{997 y\alpha\Delta x}{1000x + 997\alpha\Delta x} \right) \left(\frac{ x + \alpha\Delta x}{y - \frac{997 y \alpha \Delta x}{1000x + 997\alpha\Delta x}} \right)$

reduces to: 

$\alpha^2 \frac{997 \Delta x^2}{1000x} + \alpha\frac{1997\Delta x}{1000} - \Delta x = 0$

#### Now, solve for $\alpha$, and we can calculate the correct distribution using ```calc_deposit_dist``` 

In [ ]:
def calc_deposit_portion(lp, token_in, dx):

    tokens = lp.factory.token_from_exchange[lp.name]
    if(token_in.token_name == lp.token0):
        tkn_supply = lp.get_virtual_reserve(tokens[lp.token0])
    else:    
        tkn_supply = lp.get_virtual_reserve(tokens[lp.token1])
            
    a = 997*(dx**2)/(1000*tkn_supply)
    b = dx*(1997/1000)
    c = -dx

    alpha = -(b - np.sqrt(b*b - 4*a*c)) / (2*a)
    return alpha 

In [ ]:
alpha = calc_deposit_portion(lp, tkn, s_in)
print('The correct swap distrbution (for step 1) is {}'.format(alpha))

#### Now, check against our reduced quadratic, and we should expect to get 0

In [ ]:
#alpha = 0.49950915
997*(alpha**2)*(s_in**2)/(1000*x) + alpha*s_in*(1997/1000) - s_in

#### Finally, lets run through the steps to a ```SwapDeposit``` and compare above

In [ ]:
tkn = ERC20("TKN", "0x111")
dai = ERC20("DAI", "0x09")

exchg_data = UniswapExchangeData(tkn0 = tkn, tkn1 = dai, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("TKN pool factory", "0x2")
lp = factory.deploy(exchg_data)
lp.initialize(init_price)
out_v3 = lp.mint(user_nm, lwr_tick, upr_tick, 3162.2776601683795)

print('***\nInitial LP\n***')
lp.summary()

s_in = 100
alpha = calc_deposit_portion(lp, tkn, s_in) #+0.0000035
s_out = Swap().apply(lp, tkn, user_nm, alpha*s_in)

print('***\nLP post step 1\n***')
lp.summary()

sqrt_P = lp.slot0.sqrtPriceX96/2**96
balance1 = abs(s_out[2]) 
liq = balance1/sqrt_P 

out = lp.mint(user_nm, lwr_tick, upr_tick, liq)

print('***\nLP post step 2\n***')
lp.summary() 

print('Given {} initial TKN:'.format(s_in))
print('  (step 1) {} TKN must first get swapped for {} '.format(alpha*s_in, abs(s_out[2])))
print('  (step 2) The received TKN gets deposited along with the remaining {} TKN'.format(out[0]))

print('\nTotal deposited is {:.6f} + {:.6f} = {:.6f} TKN:'.format(alpha*s_in,  out[0], alpha*s_in + out[0]))


#### Finally, let's check when our solution is integrated into ```SwapDeposit```

In [ ]:
tkn = ERC20("TKN", "0x111")
dai = ERC20("DAI", "0x09")

exchg_data = UniswapExchangeData(tkn0 = tkn, tkn1 = dai, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("TKN pool factory", "0x2")
lp = factory.deploy(exchg_data)
lp.initialize(init_price)
out_v3 = lp.mint(user_nm, lwr_tick, upr_tick, 3162.2776601683795)
lp.summary()

s_in = 100
dep = SwapDeposit().apply(lp, tkn, user_nm, s_in, lwr_tick, upr_tick)
lp.summary()
dep

In [ ]:
tkn = ERC20("TKN", "0x111")
dai = ERC20("DAI", "0x09")
exchg_data = UniswapExchangeData(tkn0 = tkn, tkn1 = dai, symbol="LP", address="0x011")

factory = UniswapFactory("TKN pool factory", "0x2")
lp = factory.deploy(exchg_data)
lp.add_liquidity(user_nm, 10000, 1000, 10000, 1000)
lp.summary()

s_in = 100
dep = SwapDeposit().apply(lp, tkn, user_nm, s_in)
lp.summary()